## 1、数据准备与处理

1. 需要准备什么数据，取决于你希望模型完成哪些功能
2. 数据尽可能覆盖各种可能出现的情况

例如：你要模型识别地址，训练数据就要尽可能覆盖地址的各种表达形式和去一个地点的各种说法
   - <font color='blue'>我要去</font><font color='red'>天安门</font>
   - <font color='red'>太古大厦A座</font><font color='blue'>怎么走</font>
   - <font color='blue'>导航到</font><font color='red'>东方东路19号</font><font color='green'>途经</font><font color='red'>Joy Cafe</font>

### 数据采集

- 自然来源（如业务日志）：真实数据
- Web 抓取：近似数据
- 人造

### 数据标注

- 专业标注公司
  - 定标准，定验收指标
  - 预标注
  - 反馈与优化
  - 正式标注
  - 抽样检查：合格->验收；不合格->返工
- 众包
  - 定标准，定检验指标
  - 抽样每个工作者的质量
  - 维系高质量标注者社区
- 主动学习：通过模型选择重要样本，由专家标注，再训练模型
- 设计产品形态，在用户自然交互中产生标注数据（例如点赞、收藏）

### 数据清洗

- 去除不相关数据
- 去除冗余数据（例如重复的样本）
- 去除误导性数据（业务相关）

### 样本均衡性

- 尽量保证每个标签（场景/子问题）都有足够多的训练样本
- 每个标签对应的数据量尽量相当
  - 或者在保证每个标签样本充值的前提下，数据分布尽量接近真实业务场景的数据分布
- 数据不均衡时的策略
  - 数据增强：为数据不够类别造数据：（1）人工造；（2）通过模板生成再人工标注；（3）由模型自动生成（再人工标注/筛选）
  - 数据少的类别数据绝对数量也充足时，Downsample 一般比 Upsample 效果好
  - 实在没办法的话，在训练 loss 里加权（一般不是最有效的办法）
- 根据业务属性，保证其他关键要素的数据覆盖，例如：时间因素、地域因素、用户年龄段等

### 数据集构建

- 数据充分的情况下
  - 切分训练集（训练模型）、验证集（验证超参）、测试集（检验最终模型+超参的效果）
  - 以随机采样的方式保证三个集合的数据分布一致性
  - 在以上三个集合里都尽量保证各个类别/场景的数据覆盖
- 数据实在太少
  - 交叉验证


## 2、LLM 输出时每个 Token 的概率是怎么得到的

<img src="lmhead.png" width=800px>

## 3、多卡训练

### 启动脚本

单机多卡

```sh
torchrun
    --standalone
    --nnodes=1
    --nproc-per-node=$NUM_OF_GPUS
    train.py (--args ...)
```

多机多卡

```sh
torchrun
    --nnodes=$NUM_NODES
    --nproc-per-node=$NUM_OF_GPUS_PER_NODE
    --max-restarts=3
    --rdzv-id=$JOB_ID
    --rdzv-backend=c10d
    --rdzv-endpoint=$HOST_NODE_ADDR
```

代码样例

```python
import torch
import torch.distributed
from datasets import load_dataset
import os
from transformers import GPT2TokenizerFast, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
import multiprocessing
import random

MODEL_NAME = "gpt2"
SEED = 42
MAX_LENGTH = 1024
MIN_LENGTH = 4
VOCAB_SIZE = 50257

# 定义数据处理函数
def prepare_train_features(examples):
    # 略过空行
    examples["nonempty_text"] = [
        d.strip() for d in examples["text"] if len(d.strip()) > 0
    ]

    # Convert the tokens into ids using the trained tokenizer

    tokenized_example = tokenizer(
        examples["nonempty_text"],
        truncation=True,
        max_length=MAX_LENGTH*100,
    )

    # 模型输出的字段
    examples["input_ids"] = []
    examples["attention_mask"] = []

    del examples["text"]
    del examples["nonempty_text"]

    for input_ids, attention_mask in zip(tokenized_example["input_ids"], tokenized_example["attention_mask"]):

        trunc_ids = input_ids[:min(len(input_ids), MAX_LENGTH)]
        trunc_mask = attention_mask[:min(len(attention_mask), MAX_LENGTH)]

        # 把长句切成MAX_LENGTH长度的片段, 最后一段如果小于MIN_LENGTH则忽略
        while len(trunc_ids) > MIN_LENGTH:
            trunc_len = len(trunc_ids)
            if trunc_len < MAX_LENGTH:
                examples["input_ids"].append(
                    trunc_ids+[tokenizer.pad_token_id]*(MAX_LENGTH-trunc_len))
                examples["attention_mask"].append(
                    trunc_mask+[0]*(MAX_LENGTH-trunc_len))
            else:
                examples["input_ids"].append(trunc_ids)
                examples["attention_mask"].append(trunc_mask)

            input_ids = input_ids[trunc_len:]
            attention_mask = attention_mask[trunc_len:]

            trunc_ids = input_ids[:min(len(input_ids), MAX_LENGTH)]
            trunc_mask = attention_mask[:min(len(attention_mask), MAX_LENGTH)]

    examples['labels'] = examples['input_ids'].copy()

    return examples


# 开启多开训练模式
# 现在pytorch已经把这部封装了，不用手工写了
# torch.distributed.init_process_group(
#    backend='nccl', init_method="env://", rank=args.local_rank, world_size=args.word_size)
# torch.cuda.set_device(args.local_rank)

# 自动下载openwebtext数据集，展开前几十GB，展开成arrow格式大约500G
raw_datasets = load_dataset("openwebtext", split="train")
# 这里只用1%的数据作为测试集，否则每次dev时间很长
raw_datasets = raw_datasets.train_test_split(test_size=0.01)

raw_train_dataset = raw_datasets["train"]
raw_valid_dataset = raw_datasets["test"]

transformers.set_seed(args.seed)

# 定义tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
# 获取CPU核数（用于数据加载线程数）
num_proc = multiprocessing.cpu_count()

if torch.distributed.get_rank() > 0:
    # 主进程加载数据，其它进程等待从缓存加载arrow文件"
    torch.distributed.barrier()

tokenized_train_dataset = raw_train_dataset.map(
    prepare_train_features,
    batched=True,
    num_proc=num_proc
)

tokenized_valid_dataset = raw_valid_dataset.map(
    prepare_train_features,
    batched=True,
    num_proc=num_proc
)

if torch.distributed.get_rank() == 0:
    # 主进程加载数据结束
    torch.distributed.barrier()

# 定义数据校准器（自动生成batch）
collater = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, return_tensors="pt"
)

# 定义GPT-2模型config
model_config = GPT2Config(vocab_size=VOCAB_SIZE,
                          max_position_embeddings=MAX_LENGTH, return_dict=True)
# 定义模型（此处参数随机初始化）
model = GPT2LMHeadModel(config=model_config)

training_args = TrainingArguments(
    output_dir="./my_model",        # checkpoint保存路径
    evaluation_strategy="steps",    # 每N步做一次eval
    overwrite_output_dir=True,
    num_train_epochs=1,             # 训练epoch数
    per_device_train_batch_size=8,  # 每张卡的batch大小
    gradient_accumulation_steps=20,   # 累加几个step做一次参数更新
    per_device_eval_batch_size=16,  # evaluation batch size
    logging_steps=1000,             # 每1000步eval一次
    save_steps=1000,                # 每1000步保存一个checkpoint
    learning_rate=1e-3,             # 学习率
    warmup_steps=2000,              # 预热（可选）
    optim="adamw_hf",               # 求解器（默认）
)

# 定义训练器
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collater,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
)

# 开始训练
trainer.train()

if trainer.is_world_process_zero():
    # 避免每个进程保存一次
    trainer.save_model()
    
```


## 4、GPU 利用率

### 常见的造成 GPU 利用率低的原因


本质是**CPU 的计算**或**I/O**的环节耗时长，导致 GPU 利用率上不去


### 数据加载与处理的耗时

Dataloader 的几个相关参数


- num_workers：线程数
- prefetch_factor：每个 worker 提前加载样本数，设置不合理可能会让 CPU 处理时 GPU 空闲
- pin_memory：直接将数据映射到 GPU 的相关内存块上，省掉一点数据传输时间

另外，数据处理函数逻辑太复杂也影响资源利用率，建议减少 for/while 循环。


### 减少 I/O 操作的耗时

- 模型保存不宜太频繁(--save_steps 参数)
- 日志打印、指标上报、进度上报等不宜太频繁
- 存储介质对时延影响也很明显
  - 本地存储介质性能：SSD > ceph > cfs-1.5 > hdfs > mdfs
  - 网络存储：数据与计算最好在同城；排查路由、网络带宽等其它因素。
- 数据不宜分成太多小文件，会影响 I/O 性能（主要是图像处理场景）
- 分布式训练时要使用 DistributedDataParallel （Pytorch）
- 多机训练要启用 GDRDMA （英伟达的远程直接显存访问机制）


### 其它 CPU 计算耗时

主要是 loss 计算和 metric 计算的复杂度（常见的问题不涉及此处，主要是自定义 loss 或 metric 需要注意这个问题）
